In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

C:\Users\Jason\AppData\Local\Temp\ipykernel_22916\3187316332.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#pip install scipy

In [3]:
# Initial imports.
import numpy as np                          # for array operations
import pandas as pd                          # for working with DataFrames
import matplotlib.pyplot as plt        # for data visualization
%matplotlib inline                               
import seaborn as sns
from pathlib import Path
import tensorflow as tf


#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder                # transform categorical features into numerical dummy features (training)
from sklearn.model_selection import train_test_split              # for splitting the data
from sklearn.metrics import mean_squared_error                   # for calculating average
from sklearn.ensemble import RandomForestRegressor         # for building the model
from sklearn.metrics import r2_score                                        # evaluate performance of a linear regression model
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

ImportError: DLL load failed while importing _ufuncs: The specified procedure could not be found.

In [ ]:
# Loading data
file_path = ("causes_of_death.csv")
death_df = pd.read_csv(file_path)
death_df.head()

In [ ]:
# Generate our categorical variable list
death_cat = death_df.dtypes[death_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
death_df[death_cat].nunique()

In [ ]:
death_df.dtypes

In [ ]:
death_df.info()

In [ ]:
# Check the unique value counts to see if binning is required
death_df['Cause Name'].value_counts()

In [ ]:
# Check the unique value counts to see if binning is required
death_df['State'].value_counts()

In [ ]:
# Drop columns that are not useful
death_df =  death_df.drop('Cause Name 113', axis =1)

In [ ]:
death_df.head()

In [ ]:
death_df_encoded = pd.get_dummies(death_df, columns=["Cause Name", "State"])
death_df_encoded.head()

In [ ]:
death_df.dtypes

In [ ]:
# Define the features data set.
X = death_df_encoded.copy()
#X = X.drop(["Death Rate","Deaths", "Cause Name_All causes", "Cause Name_Cancer", "Cause Name_Heart disease"], axis=1)
X = X.drop(["Death Rate","Deaths", "Cause Name_All causes"], axis=1)
X.head()


In [ ]:
# Define the target data set.
y = death_df["Death Rate"].ravel()
y[:5]

In [ ]:
# Split the dataset into training and testing set (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)


In [ ]:
# Define the Estimator and the Parameter Ranges
dt = RandomForestRegressor()
number_of_iterations = 20
max_leaf_nodes = [2, 3, 4, 5, 6, 7]
min_samples_split = [5, 10, 20, 50]
max_depth = [5,10,15,20]
max_features = [3,4,5]
n_estimators = [50, 100, 200]

# Define the param distribution dictionary
param_distributions = dict(max_leaf_nodes=max_leaf_nodes, 
                           min_samples_split=min_samples_split, 
                           max_depth=max_depth,
                           max_features=max_features,
                           n_estimators=n_estimators)

# Build the gridsearch
grid = RandomizedSearchCV(estimator=dt, 
                          param_distributions=param_distributions, 
                          n_iter=number_of_iterations, 
                          cv = 5)

grid_results = grid.fit(x_train, y_train)

# Summarize the results in a readable format
print("Best params: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_df = pd.DataFrame(grid_results.cv_results_)

In [ ]:
# Initializing the Random Forest Regression model with 128 decision trees
model = RandomForestRegressor(n_estimators =128, max_features = 'sqrt', max_depth = 7, random_state = 0)


In [ ]:
# Fitting the Random Forest Regression model to the data
model.fit(X_train, y_train)

In [ ]:
# Predicting the target values of the test set
y_pred = model.predict(X_test)

To calculate the loss between the actual target values in the testing set and the values predicted by the model, a cost function called the Root Mean Square Error (RMSE) is used.

The RMSE  indicates how close the actual data points are to the model’s predicted values. When RMSE value is low it indicates a better fit and is a good measure for determining the accuracy of the model’s predictions.

In [ ]:
# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)),'.3f'))
print("\nRMSE:\n",rmse)

# RMSE for training....verifying if data leakage was occuring
#rmse = float(format(np.sqrt(mean_squared_error(y_train, model.predict(X_train))),'.3f'))
#print("\nRMSE:\n",rmse)

R² score indicates how will our model is fitte to the data by comparing it to the average line of the dependent variable (y or target). A score closer to 1, implies our model preformed well verses if score is further from 1, implying our model did not perform well

In [ ]:
# R² score.....indicates that 20% of the variancce in the causes of death can be explained??? 
r2_score(y_test, y_pred)

In [ ]:
model.feature_importances_

In [ ]:
df_feature_importance = pd.DataFrame({'feature': X.columns, 'feature_importance':model.feature_importances_})
df_feature_importance.head()

In [ ]:
df_feature_importance.sort_values('feature_importance')